In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from glob import glob
import itertools
from pao_file_utils import parse_pao_file
from sympy.physics.quantum.cg import CG

In [3]:
#https://github.com/cp2k/cp2k/blob/master/src/common/spherical_harmonics.F
def Y_l(r, l):
    """Real Spherical Harmonics"""
    assert r.shape[-1] == 3

    if l < 0:
        raise Exceptoin("Negative l value")
    elif l == 0:
        return np.sqrt(1.0 / (4.0 * np.pi))
    elif l == 1:
        pf = np.sqrt(3.0 / (4.0 * np.pi))
        return pf * r
    elif l == 2:
        x = r[..., 0]
        y = r[..., 1]
        z = r[..., 2]
        result = np.zeros(5)
        # m = 2
        pf = np.sqrt(15.0 / (16.0 * np.pi))
        result[0] = pf * x**2 - y**2
        # m = 1
        pf = np.sqrt(15.0 / (4.0 * np.pi))
        result[1] = pf * z * x
        # m = 0
        pf = np.sqrt(5.0 / (16.0 * np.pi))
        result[2] = pf * (3.0 * z**2 - 1.0)
        # m = -1
        pf = np.sqrt(15.0 / (4.0 * np.pi))
        result[3] = pf * z * y
        # m = -2
        pf = np.sqrt(15.0 / (16.0 * np.pi))
        result[4] = pf * 2.0 * x * y
    return result

In [72]:
def convolute(coords, kinds, central_atom):
    natoms = coords.shape[0]
    assert coords.shape[1] == 3

    integrals = []
    for l in range(3):  #max_l = 2
        for sigma in [0.5, 1.0, 2.0, 3.0, 4.0]:
            for ikind in sorted(kinds):
                integrals.append(np.zeros(2*l + 1))
                for iatom in range(natoms):
                    if atom2kind[iatom] == ikind and iatom != central_atom:
                        r = coords[central_atom] - coords[iatom]
                        angular_part = Y_l(r, l)
                        radial_part = np.exp(- np.dot(r,r) / sigma**2)
                        integrals[-1] += radial_part * angular_part
    return integrals

In [95]:
cg_cache = dict()

def get_clebsch_gordan_coefficients(li, lj, lo):
    global cg_cache
    key = (li, lj, lo)
    if key not in cg_cache:
        coeffs = np.zeros(shape=(2*li+1, 2*lj+1, 2*lo+1))
        for mi in range(-li, li+1):
            for mj in range(-lj, lj+1):
                for mo in range(-lo, lo+1):
                    # https://docs.sympy.org/latest/modules/physics/quantum/cg.html
                    cg = CG(li, mi, lj, mj, lo, mo).doit() #TODO: cache and vectorize
                    coeffs[mi+li, mj+lj, mo+lo] = cg
        cg_cache[key] = coeffs
    return cg_cache[key]

In [96]:
def combinations(channels, max_l):
    """ returns all possbile combinations of input channels up to given max_l """
    output_channels = list()
    for channel_i, channel_j in itertools.combinations_with_replacement(channels, 2):
        assert len(channel_i.shape) == len(channel_j.shape) == 1
        li = (channel_i.size - 1) // 2
        lj = (channel_j.size - 1) // 2
        # There li + lj possible ways to combine the two channels.
        # We do all of them up to a max_l.
        for lo in range(min(max_l, li+lj) + 1): # l of output
            channel_o = np.zeros(2*lo+1)
            cg = get_clebsch_gordan_coefficients(li, lj, lo)
            channel_o = np.einsum("i,j,ijo->o", channel_i, channel_j, cg)
            output_channels.append(channel_o)
    return output_channels

In [104]:
samples = []
pao_files = sorted(glob("2H2O_MD/frame_*/2H2O_pao44-1_0.pao"))
kinds, atom2kind, coords, xblocks = parse_pao_file(pao_files[0])


initial_channels = convolute(coords, kinds, 0)
print("initial_channels: ", len(initial_channels))

comb_channels = combinations(initial_channels, max_l=2)
print("channel combinations: ", len(comb_channels))

s_channels = [c for c in comb_channels if c.size == 1]
print("s channels: ", len(s_channels))

#TODO: build pao basis vector from linear combinations of comb_channels using coefficients w
#TODO: build NN that maps takes s_channels as input and outputs coefficients w

initial_channels:  30
channel combinations:  1185
s channels:  465
